# H.02 | Penguins

Our adventure begins with a database (`penguins.db`) containing information about three species of penguins: Adelie, Chinstrap, and Gentoo. The database includes various measurements such as the penguin's bill length, bill depth, flipper length, and body mass. Our goal is to explore this dataset, analyze the data, and gain insights into the penguins' characteristics.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display, Markdown

## Basic SQL.

Let's get some practice querying SQL to obtain your dataset. The dataset is stored in a local SQLite database named `penguins.db`. The simplest SQL query is a `SELECT` statement that retrieves data from a table. Please implement the `get_sql_query_basic` function in `basic_statistics.py` that returns a SQL query that retrieves all columns and rows from the penguins table.

`read_sqlite_db_to_pandas` is a function written for you. It queries the `penguins.db` database and returns a pandas DataFrame. You can use this function to test your SQL query.

In [ ]:
from basic_statistics import read_sqlite_db_to_pandas, get_sql_query_basic

basic_query = get_sql_query_basic()
read_sqlite_db_to_pandas(basic_query)

## Challenge SQL

Please implement the `get_sql_query_challenge` function in `basic_statistics.py`. This is a slightly more challenging SQL query -- please ensure you follow the instructions in the docstring.

In [ ]:
from basic_statistics import get_sql_query_challenge

challenge_query = get_sql_query_challenge()
read_sqlite_db_to_pandas(challenge_query)

## Exploratory Data Analysis

Download the dataset and take a look using descriptive exploratory data analysis. Note: your basic_query from `get_sql_query_basic` should return the entire penguins table.

In [ ]:
# Let's get our full dataset from the db.
df = read_sqlite_db_to_pandas(basic_query)

# Visualize the first few rows of the dataset (default n = 5)
display(Markdown("-------------\n### Top of the Dataset: `df.head()`\n**Note: The dataset contains 344 rows and 8 columns.**"))
display(df.head())

# This method provides a concise summary of the dataset, including the number of non-null values in each column.
display(Markdown("-------------\n### Detailed Information: `df.info()`\n**Note: We have some missing values in the dataset**"))
display(df.info())

# Descriptive statistics for numerical columns
display(Markdown("-------------\n### Descriptive Statistics: `df.describe()`\n"))
display(df.describe())
display(Markdown("-------------"))

## Data Cleaning

You can see from the df.info() call that we are missing values. Clean the data by handling missing values. Let's assume for now that the missing values in the float64 columns are missing at random.

We will fill in the float64 columns with the following methods (located in `basic_statistics.py`): 

- Mean Imputation (**fill_float64_cols_with_mean**)
- Random Imputation (**fill_float64_cols_with_random**)

One of the main goals for this task is to gain familiarity with Pandas. Please note, you are only expected to fill in the functions that have not been implemented yet -- these functions will focus on getting you to a place where you are comfortable with Pandas. [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) is a wonderful tool for data scientists, and familiarity with it will help your future endeavors!

You will be asked to write three functions in `basic_statistics.py`:

1. **get_float64_column_names**

    Working with pandas dataframes requires you to know the column names and their data types. This function should return the names of the float64 columns in the dataset as a list of strings.

2. **get_missing_value_indices**

    Working with missing values in pandas can be tricky, especially since missing values can be notated in many ways. This function should return the row indices of the missing values in the float64 columns as a list of ints.

3. **drop_missing_values**

    Often, the best recourse is to drop the missing values. This function should drop any rows containing missing values (regardless of the column type).

In [ ]:
from basic_statistics import fill_float64_cols_with_mean, fill_float64_cols_with_random_sample

mean_imputed_df = fill_float64_cols_with_mean(df.copy())
random_imputed_df = fill_float64_cols_with_random_sample(df.copy())

display(Markdown("### Original"))
display(df.head())

display(Markdown("### Mean Imputed"))
display(mean_imputed_df.head())

display(Markdown("### Random Imputed"))
display(random_imputed_df.head())

### Imputation Comparison

After imputing missing values, we can visualize the data to understand the impact of our imputation.

In [ ]:
# Drop first 75 rows and fill with mean imputation (just for demonstration purposes).
copy_df = df.copy()
copy_df.loc[0:75, 'bill_length_mm'] = np.nan
mean_imputed_df = fill_float64_cols_with_mean(copy_df)

# Drop first 75 rows and fill with random imputation (just for demonstration purposes).
copy_df = df.copy()
copy_df.loc[0:75, 'bill_length_mm'] = np.nan
random_sample_imputed_df = fill_float64_cols_with_random_sample(copy_df)

# Combine the original and imputed dataframes for comparison.
mean_imputed_df["mean_imputed_bill_length_mm"] = mean_imputed_df["bill_length_mm"]
random_sample_imputed_df["random_sample_imputed_bill_length_mm"] = random_sample_imputed_df["bill_length_mm"]
imputed_df_combined = pd.concat([copy_df, mean_imputed_df, random_sample_imputed_df], axis=0)

# Plot the violin plot.
px.violin(imputed_df_combined, y=["bill_length_mm", "mean_imputed_bill_length_mm", "random_sample_imputed_bill_length_mm"], box=True, points="all", template = "plotly_white", title = "Comparing Imputation Methods for Bill Length")

## Hypotheses Testing

Form and test hypotheses about the penguins' characteristics.

Now that we have taken care of our missing data values, let's use plotly to create some visualizations! Below you will see the scatter matrix of the penguins dataset, specifically around the columns 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', and 'body_mass_g'.

What patterns can you begin to spot in our data?

In [ ]:
# Plot scatter matrix.
fig = px.scatter_matrix(df, dimensions=["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"], color="species")
fig.update_layout(title="Penguin Data", template = "plotly_white", width=900, height=900)
fig.show()

It looks like we may have an interesting relationship between flipper length and species. Let's explore this further visually.

In [ ]:
# Box plot for flipper length by species.
fig = px.box(df, x="species", y="flipper_length_mm", color="species", title="Flipper Length by Species", template="plotly_white")
fig.show()

After visualizing the data, we start to form some hypotheses.

**Hypothesis 1**: The body mass of the penguins has a strong positive correlation with the flipper length.

We will compare covariance, pearson correlation, and spearman correlation to quantify this relationship.
You will be asked to write the following functions in `basic_statistics.py`:

1. **calculate_covariance_numpy**
    - Using only basic methods in numpy, calculate the covariance between two arrays. Numpy is a tremendously useful tool in python. Learning how to look at a formula and implement it in numpy is a valuable skill. This will also help you understand the formula for covariance.
2. **calculate_pearson_correlation_numpy**
    - Using only basic methods in numpy, calculate the Pearson correlation between two arrays. This will also help you understand the formula for Pearson correlation.
3. **calculate_pearson_correlation_scipy**
    - Using the scipy library, calculate the Pearson correlation between two arrays. Scipy is a powerful library that can help you perform statistical tests and calculations.
4. **calculate_spearman_correlation_scipy**
    - Using the scipy library, calculate the Spearman correlation between two arrays. Scipy is a powerful library that can help you perform statistical tests and calculations.

**Hypothesis 2**: Chinstrap penguins have a longer flipper length than Adelie penguins.

Let's compare the flipper length of Chinstrap penguins with the flipper length of Adelie penguins. You will be asked to write the following function in `basic_statistics.py`:

1. **perform_hypothesis_test**
    - Using the scipy library, perform the appropriate hypothesis test to compare the flipper length of Chinstrap penguins with the flipper length of Adelie penguins.

2. **check_normality**
    - Using the scipy library, check the normality of the data. This will help you determine if the data is normally distributed.

3. **check_variance_homogeneity**
    - Using the scipy library, check the homogeneity of variance. This will help you determine if the variances of the two groups are equal.

### Hypothesis 1

We suspect that the body mass of the penguins has a strong positive correlation with the flipper length. Let's take a closer look!



**Covariance**

Recall that covariance is a measure of the relationship between two random variables. The covariance between two variables is positive if they tend to increase together and negative if one decreases as the other increases. The covariance is zero if the variables are unrelated. It is defined as: 

$ \text{cov}(X, Y) = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{n} $

**Pearsons**:

Pearson correlation is a measure of the linear relationship between two variables. It ranges from -1 to 1, where 1 indicates a perfect positive linear relationship, -1 indicates a perfect negative linear relationship, and 0 indicates no linear relationship. Pearson's is defined as:

$r = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2}\sqrt{\sum_i (y_i - \bar{y})^2}} $

**Spearmans**:

Spearman correlation is a non-parametric measure of the monotonic relationship between two variables. It ranges from -1 to 1, where 1 indicates a perfect positive monotonic relationship, -1 indicates a perfect negative monotonic relationship, and 0 indicates no monotonic relationship. The formula below assumes all n ranks are distinct integers. Spearmans is defined as:

$ r_s = 1 - \frac{6 \sum_i d_i^2}{n^3 - n} $ <br>
$ d_i = \text{rank}(x_i) - \text{rank}(y_i) $

In [ ]:
from basic_statistics import (calculate_covariance_numpy, calculate_pearson_correlation_numpy, 
                              calculate_pearson_correlation_scipy, calculate_spearman_correlation_scipy)

df = df.dropna()
x = df["body_mass_g"]
y = df["flipper_length_mm"]

numpy_covariance = round(calculate_covariance_numpy(x, y), 3)
numpy_pearson = round(calculate_pearson_correlation_numpy(x, y), 3)
scipy_pearson = round(calculate_pearson_correlation_scipy(x, y), 3)
scipy_spearman = round(calculate_spearman_correlation_scipy(x, y), 3)

Markdown(
    "| Metric | Value |\n" +
    "|--------|-------|\n" +
    f"| Covariance | {numpy_covariance} |\n" +
    f"| Pearson's Correlation (manual) | {numpy_pearson} |\n" +
    f"| Pearson's Correlation (scipy) | {scipy_pearson} |\n" +
    f"| Spearman's Correlation (scipy) | {scipy_spearman} |\n"
)

### Hypothesis 2

We suspect that Chinstrap penguins have a longer mean flipper length than Adelie penguins. Let's put this to the test!

Specifically, here are our null (H0) and alternative (HA) hypotheses:

- **H0**: The flipper length of Chinstrap penguins is equal to the flipper length of Adelie penguins. <br>
- **H1**: The flipper length of Chinstrap penguins is not equal to the flipper length of Adelie penguins. <br>

In [ ]:
from basic_statistics import check_normality, check_variance_homogeneity, perform_hypothesis_test

df = df.dropna()
x = df[df["species"] == "Adelie"]["flipper_length_mm"]
y = df[df["species"] == "Chinstrap"]["flipper_length_mm"]

normality_statistic_adelie, normality_p_value_adelie = check_normality(x)
normality_statistic_chinstrap, normality_p_value_chinstrap = check_normality(y)
equal_variance_statistic, equal_variance_p_value = check_variance_homogeneity(x, y)
statistic, p_value = perform_hypothesis_test(x, y)

Markdown(
    "| Test | Statistic | p-value |\n" +
    "|------|-----------|---------|\n" +
    f"| Shapiro-Wilks (Adelie) | {normality_statistic_adelie} | {normality_p_value_adelie} |\n" +
    f"| Shapiro-Wilks (Chinstrap) | {normality_statistic_chinstrap} | {normality_p_value_chinstrap} |\n" +
    f"| Levene | {equal_variance_statistic} | {equal_variance_p_value} |\n" +
    f"| Hypothesis Test | {statistic} | {p_value} |\n"
)